In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

In [2]:
@qml.qfunc_transform
def rotate_rots(tape, params):
    for op in tape.operations + tape.measurements:
        if op.name == "RX":
            if list(op.wires) == [0]:
                qml.RX(op.parameters[0] + params[0], wires=op.wires)
            else:
                qml.RX(op.parameters[0] + params[1], wires=op.wires)
        elif op.name == "RY":
            if list(op.wires) == [0]:
                qml.RY(op.parameters[0] + params[2], wires=op.wires)
            else:
                qml.RY(op.parameters[0] + params[3], wires=op.wires)
        elif op.name == "RZ":
            if list(op.wires) == [0]:
                qml.RZ(op.parameters[0] + params[4], wires=op.wires)
            else:
                qml.RZ(op.parameters[0] + params[5], wires=op.wires)
        else:
            qml.apply(op)

def circuit():
    qml.RX(np.pi / 2, wires=0)
    qml.RY(np.pi / 2, wires=0)
    qml.RZ(np.pi / 2, wires=0)
    qml.RX(np.pi / 2, wires=1)
    qml.RY(np.pi / 2, wires=1)
    qml.RZ(np.pi / 2, wires=1)

In [3]:
def optimal_fidelity(target_params, pauli_word):

    dev = qml.device("default.qubit", wires=2)

    @qml.qnode(dev)
    def target_circuit(target_params, pauli_word):
        rot_name = 'R' + pauli_word
        getattr(qml,rot_name)(phi=target_params[0],wires=0)
        qml.CRX(phi=target_params[1],wires=[0,1])
        qml.T(wires=0)
        qml.S(wires=1)
        return qml.state()

    param_x = (np.pi)/2
    def circuit(param_x):
        qml.RX(param_x, wires=0)
        qml.RY(param_x, wires=0)
        qml.RZ(param_x, wires=0)
        qml.RX(param_x, wires=1)
        qml.RY(param_x, wires=1)
        qml.RZ(param_x, wires=1)

    
    @qml.qnode(dev)
    def rotated_circuit(param_x, transform_weights):

        # apply the transform to the ansatz
        rotate_rots(transform_weights)(circuit)(param_x)
        return qml.state()
    
    def cost_function(rot_params):
        state_0 = target_circuit(target_params, pauli_word)
        state_1 = rotated_circuit(param_x, rot_params)
        FidelitY = qml.math.fidelity(state_1, state_0)
        return FidelitY

    opt = qml.GradientDescentOptimizer(stepsize=0.1)
    rot_params = np.array([0.19, 0.55, 0.86, 0.5, 0.2, 0.88], requires_grad=True)
    max_iterations = 200
    
    for n in range(max_iterations):
        rot_params = opt.step(cost_function, rot_params)
    
    Max_Fidelity = cost_function(rot_params)
    
    return Max_Fidelity

In [4]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:

    ins = json.loads(test_case_input)
    output = optimal_fidelity(*ins)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    """
    Compare solution with expected.

    Args:
            solution_output: The output from an evaluated solution. Will be
            the same type as returned.
            expected_output: The correct result for the test case.

    Raises:
            ``AssertionError`` if the solution output is incorrect in any way.
    """

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-2
    ), "Your calculated optimal fidelity isn't quite right."

In [5]:
test_cases = [['[[1.6,0.9],"X"]', '0.9502'], ['[[0.4,0.5],"Y"]', '0.9977']]

In [6]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[1.6,0.9],"X"]'...
Correct!
Running test case 1 with input '[[0.4,0.5],"Y"]'...
Correct!
